In [ ]:
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import numpy as np


scope = ['https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive']

#개인에 따라 수정 필요 - 다운로드 받았던 키 값 경로 
json_key_path = "astute-catcher-blurblur.json"	

credential = ServiceAccountCredentials.from_json_keyfile_name(json_key_path, scope)
gc = gspread.authorize(credential)


#개인에 따라 수정 필요 - 스프레드시트 url 가져오기
spreadsheet_url = "https://docs.google.com/spreadsheets/d/{sheet key}"

doc = gc.open_by_url(spreadsheet_url)

#개인에 따라 수정 필요 - 시트 선택하기 (시트명을 그대로 입력해주면 된다.)
sheet_name = "설문지 응답 시트1"
sheet = doc.worksheet(sheet_name)

#데이터 프레임 생성하기
df = pd.DataFrame(sheet.get_all_values())

#불러온 데이터 프레임 정리

columns = {
    0: 'submission_time',
    1: 'name',
    2: 'preferred_project',
    3: 'project_experience',
    4: 'competency_test_grade',
    5: 'competency_test_target_grade',
    6: 'close_friends'
}


df.rename(columns=columns, inplace = True)
df.drop(df.index[0], inplace=True)

df.drop('submission_time', axis=1, inplace=True)


df.head()

In [ ]:
def calculate_score(row):
    score = 0
    
    # 선호 프로젝트 점수 계산 50%
    project = str(row['project_experience']).lower()
    if '프론트엔드' in project or 'frontend' in project:
        score += 15
    if '백엔드' in project or 'backend' in project:
        score += 15
    if 'db' in project or '디비' in project:
        score += 5
    if '디자인' in project:
        score += 8
    if '기획' in project:
        score += 7
        
    # 역량테스트 등급 점수 계산 30%
    grade = str(row['competency_test_grade'])
    if grade == 'A':
        score += 25
    elif grade == 'B':
        score += 30
        
    # 역량테스트 등급 점수 계산 20%
    grade = str(row['competency_test_target_grade'])
    if grade == 'IM':
        score += 10
    elif grade == 'A':
        score += 15
    elif grade == 'B':
        score += 20
    
    score /= 100
    return round(score, 2)

df['score'] = df.apply(calculate_score, axis=1)

# 점수 순위 계산
df['rank'] = df['score'].rank(method='dense', ascending=False)

# 점수가 높은 순서대로 정렬
df_sorted = df.sort_values('score', ascending=False)

print("\n=== 점수 순위 (높은 순) ===")
print(df_sorted[['name', 'score', 'rank']])
# print(df[['name', 'preferred_project', 'competency_test_grade', 'competency_test_target_grade']])



In [ ]:
def calculate_score_compatibility(score1, score2, target_sum):
    # 점수 차이가 적을수록 높은 호환성 (35% 가중치)
    w = 0.35
    score_diff = abs((score1 + score2) - target_sum)
    max_possible_diff = target_sum  # 최대 가능한 차이
    score_compatibility = 1 - (score_diff / max_possible_diff)
    return score_compatibility * w

def calculate_friend_compatibility(person1, person2, df):
    # 친구 관계 호환성 (40% 가중치)
    w = 0.4
    friends1 = str(df[df['name'] == person1]['close_friends'].values[0])
    friends2 = str(df[df['name'] == person2]['close_friends'].values[0])
    
    no_preference1 = '상관없음' in friends1.replace(' ', '') or '상관 없음' in friends1
    no_preference2 = '상관없음' in friends2.replace(' ', '') or '상관 없음' in friends2
    
    if no_preference1 and no_preference2:
        return w
    elif no_preference1:
        return w if person1 in friends2 else 0
    elif no_preference2:
        return w if person2 in friends1 else 0
    else:
        return w if (person2 in friends1 and person1 in friends2) else 0

def calculate_project_compatibility(person1, person2, df):
    # 프로젝트 주제 호환성 (25% 가중치)
    w = 0.25
    proj1 = str(df[df['name'] == person1]['preferred_project'].values[0])
    proj2 = str(df[df['name'] == person2]['preferred_project'].values[0])
    
    if '상관 없음' in proj1 or '상관없음' in proj1:
        return w
    if '상관 없음' in proj2 or '상관없음' in proj2:
        return w
        
    # 프로젝트 목록을 집합으로 변환
    proj1_set = set(p.strip() for p in proj1.split(','))
    proj2_set = set(p.strip() for p in proj2.split(','))
    
    # 공통 프로젝트가 있으면 만점
    return w if proj1_set & proj2_set else 0
def calculate_team_compatibility(team, df):
    total_compatibility = 0
    pairs = 0
    
    # 전체 학생들의 평균 점수를 target_sum으로 사용
    total_class_score = df['score'].sum()
    avg_score_per_person = total_class_score / len(df)
    target_sum = avg_score_per_person * 2  # 2인 1팀 기준
    # target_sum = 1
    for i in range(len(team)):
        for j in range(i+1, len(team)):
            score1 = df[df['name'] == team[i]]['score'].values[0]
            score2 = df[df['name'] == team[j]]['score'].values[0]
            
            # 각 호환성 점수를 0~1 사이로 정규화
            score_comp = calculate_score_compatibility(score1, score2, target_sum)
            friend_comp = calculate_friend_compatibility(team[i], team[j], df)
            project_comp = calculate_project_compatibility(team[i], team[j], df)
            
            # 각 호환성 점수의 평균을 계산
            pair_compatibility = (score_comp + friend_comp + project_comp)
            total_compatibility += pair_compatibility
            pairs += 1
            
    # 팀 전체의 평균 호환성 점수 반환
    return total_compatibility / pairs if pairs > 0 else 0

def create_balanced_teams(df_sorted, df):
    teams = []
    used_people = set()
    scores = df_sorted[['name', 'score']].values.tolist()
    n = len(scores)
    target_team_sum = sum(score for _, score in scores) / (n // 2)
    # target_team_sum = 1
    
    while len(used_people) < n:
        available = [(name, score) for name, score in scores if name not in used_people]
        if len(available) < 2:
            break
            
        first_person = available[0]
        best_compatibility = -1
        best_partner = None
        
        # 가장 호환성이 높은 파트너 찾기
        for second_person in available[1:]:
            if second_person[0] in used_people:
                continue
                
            # 각 기준별 호환성 계산
            score_comp = calculate_score_compatibility(first_person[1], second_person[1], target_team_sum)
            friend_comp = calculate_friend_compatibility(first_person[0], second_person[0], df)
            project_comp = calculate_project_compatibility(first_person[0], second_person[0], df)
            
            # 총 호환성 점수
            total_compatibility = score_comp + friend_comp + project_comp
            
            if total_compatibility > best_compatibility:
                best_compatibility = total_compatibility
                best_partner = second_person
        
        if best_partner:
            teams.append([first_person[0], best_partner[0]])
            used_people.add(first_person[0])
            used_people.add(best_partner[0])
    
    # 홀수 인원 처리
    remaining = [name for name, _ in scores if name not in used_people]
    if remaining:
        # 마지막 한 명이 남았다면 가장 호환성 높은 팀에 추가
        last_person = remaining[0]
        best_team_compatibility = -1
        best_team_idx = -1
        
        for i, team in enumerate(teams):
            avg_compatibility = (
                calculate_friend_compatibility(last_person, team[0], df) +
                calculate_friend_compatibility(last_person, team[1], df) +
                calculate_project_compatibility(last_person, team[0], df) +
                calculate_project_compatibility(last_person, team[1], df)
            ) / 2
            
            if avg_compatibility > best_team_compatibility:
                best_team_compatibility = avg_compatibility
                best_team_idx = i
        
        if best_team_idx != -1:
            teams[best_team_idx].append(last_person)
    
    return teams, target_team_sum

# 결과 출력
teams, target_sum = create_balanced_teams(df_sorted, df)
print(f"\n=== 팀 구성 결과 (목표 팀 점수: {target_sum:.1f}) ===")
for i, team in enumerate(teams, 1):
    team_scores = [df[df['name'] == person]['score'].values[0] for person in team]
    team_total = sum(team_scores)
    team_compatibility = calculate_team_compatibility(team, df)
    team_info = ' - '.join([
        f"{person}(점수:{score}, 선호:{df[df['name'] == person]['preferred_project'].values[0]})"
        for person, score in zip(team, team_scores)
    ])
    print(f"팀 {i}: {team_info}")
    print(f"   총점: {team_total:.1f}, 목표점수와의 차이: {(team_total - target_sum):.1f}")
    print(f"   팀 호환성 점수: {team_compatibility:.2f}")